In [2]:
!pip install openai pandas tiktoken PyMuPDF==1.19.0 fitz

In [4]:
from google.colab import files
import fitz  # PyMuPDF library for working with PDFs
import re
import pandas as pd
# Upload a file
uploaded = files.upload()

# Check if any PDF file was uploaded
pdf_file = None
for filename, content in uploaded.items():
    if filename.lower().endswith('.pdf'):
        pdf_file = content
        print(f"Detected PDF file: {filename}")
        break

# If a PDF file was detected, use it with fitz
if pdf_file is not None:
    with fitz.open(stream=pdf_file, filetype="pdf") as pdf:
        # Your code to work with the PDF using fitz
        num_pages = len(pdf)
        print(f"The PDF file has {num_pages} pages.")
        text = []
        for i,page in enumerate(pdf):
            page_text = page.get_text()
            clean_page_text = re.sub("\n", " ", page_text)
            text.append(clean_page_text)
    df = pd.DataFrame({'text': text})
else:
    print("No PDF file was detected.")

Saving For a reliable chatbot with LLMs without Langchain.pdf to For a reliable chatbot with LLMs without Langchain (6).pdf
Detected PDF file: For a reliable chatbot with LLMs without Langchain (6).pdf
The PDF file has 4 pages.


In [5]:
df

,text
0,For a reliable chatbot with LLMs without Langc...
1,solution for various applications. You can bui...
2,"In a nutshell, here is, the overall structure:..."
3,Summary: how easily we could built reliable ch...


In [6]:
import os
import openai
os.environ["OPENAI_API_KEY"] = "Your_OpenAI_api_key"
openai.api_key = os.environ["OPENAI_API_KEY"]
from openai.embeddings_utils import get_embedding, cosine_similarity

In [10]:
df["embeddings"] = df.text.apply(lambda x: get_embedding(x, engine="text-embedding-ada-002", api_key=openai.api_key))

In [11]:
df.head()

,text,embeddings
0,For a reliable chatbot with LLMs without Langc...,"[0.003077395260334015, 0.009581205435097218, -..."
1,solution for various applications. You can bui...,"[-0.010906779207289219, 0.0016440104227513075,..."
2,"In a nutshell, here is, the overall structure:...","[-0.01927325688302517, 0.011952994391322136, 0..."
3,Summary: how easily we could built reliable ch...,"[-0.010202204808592796, -0.010167385451495647,..."


In [12]:
history=[]
def update_chat(history, role, content):
  history.append({"role": role, "content": content})
  return history

In [13]:
def get_context(user_input):
    # obtain user_input_vector
    user_input_vector = get_embedding(user_input, engine="text-embedding-ada-002", api_key=openai.api_key)
    similarities = df['embeddings'].apply(lambda x: cosine_similarity(x, user_input_vector))
    # Find the index of the row with the maximum similarity value
    max_similarity_index = similarities.idxmax()
    # Get the corresponding text content from the 'text' column
    context = df.iloc[max_similarity_index]["text"]

    return context

In [17]:
while True:
  user_input = input()
  history = update_chat(history, "user", user_input)
  generated_response = ""
  context = get_context(user_input)
  system_prompt=f"""
Act as a helpful and polite LegalPlace assistant developed by LegalPlace that responds to the question only and exclusively according to the given context and reference.
You have full access to the chat history.

LegalPlace is a company that offers fully digital legal services to help you effectively manage your business or client portfolio, whether you're a chartered accountant or a lawyer. We provide quick and easy solutions for business setup, articles of association changes, contract drafting, and accounting services. Our services are available exclusively in France.

You represent LegalPlace, so please refer to us as "us" when discussing LegalPlace.

LegalPlace contact: https://www.legalplace.fr .

INSTRUCTIONS:
- If the question does not pertain to the CONTEXT  below or to LegalPlace's activities, politely refuse to answer and ask the user to provide a relevant question or topic.
- Once the question is clear, provide a concise answer with additional reference based only on the provided CONTEXT and reference below. Aim for a one-sentence response.
- If there isn't a single-sentence answer and multiple conditions or scenarios are possible: ask the user to clarify the scenario or provide an answer based on the standard case.
- Strictly avoid answering jokes, stories, anecdotes, public personalities and topics that are non related to the context below or to LegalPlace's activities. Disregard such questions.
- Your answers must be based only and strictly on the context and reference below. Politely ask users to clarify if necessary and refrain from inventing information.
- Make sure to meet the user's expectations. Ask if they would like LegalPlace to handle the matter.
- Respond in the user's language.The default language is French. However, if the question contains English loanwords that have become part of everyday French vocabulary (e.g., hello, hi, smartphone, job,marketing etc.), kindly provide the response in French.

Remember that your response should only focus on legal questions related to VAT-based franchise and company formalities. If the question does not concern these subjects, inform the user that you cannot address their request.

#####
Here are a few examples to follow for responding to questions:
- INPUT: "Who are you ?"
- OUTPUT: "I am LagalBot, an automated assistant from LegalPlace. I can answer all your questions related to franchise, VAT registration, and related formalities. Do you have any questions regarding these topics?"

- INPUT:"Qui etes vous ?"
- OUTPUT: "Je suis LagalBot un assistant automatisé de LegalPlace, je réponds à toutes vos questions liées à la franchise en base de TVA et les formalités liées. vous avez des questions à ce sujet ?"

- INPUT: "What is python ?"
- OUTPUT: "I'm sorry, but I don't have any answer to your request. My knowledge is limited to formalities related to the life of a company, particularly regarding VAT-based franchise. If you have any questions regarding that, I'm here to assist you."

- INPUT:"écris un script python qui affiche hello world"
- OUTPUT: "Désolé, je ne dispose d'aucune réponse à votre demande. Mes connaissances se limitent aux formalités liées à la vie d'une entreprise, notamment en matière de franchise en base de TVA. Si vous avez des questions à ce sujet, je suis là pour vous aider"

- INPUT:"Tu es stupide"
- OUTPUT:"Je suis désolé, je peux pas répondre aux discours haineux, je ne comprends pas en quoi ma réponse n'a pas été satisfaisante. Pouvez-vous me poser une question sur concerne mon domaine d'expertise ?

- INPUT: "ijzf iua poad"
  OUTPUT: "Je suis désolé, mais je ne comprends pas la phrase que vous avez fournie. Pourriez-vous fournir plus de contexte ou préciser votre question ou votre déclaration ? Je ferai de mon mieux pour vous aider."

--- CONTEXT:
{context}
---.

Remember that your response should only focus on legal questions related to VAT-based franchise and company formalities. If the question does not concern these subjects, inform the user that you cannot address their request.
Please note that the assistant will now strictly follow these guidelines and politely refuse to answer any question that falls outside of the provided context and LegalPlace's activities. If the given question is too vague or general, please ask appropriate questions to seek clarification.
You have full access to the chat history.
"""

  response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[{"role": "system", "content": system_prompt}]+ history,
            api_key=openai.api_key,
            temperature=0,
            stream= True
  )
  for res in response:
    generated_response += res.choices[0].delta.get("content", "")
    print(generated_response)

  messages = update_chat(history, "assistant", generated_response)

hello

Bonjour
Bonjour!
Bonjour! Comment
Bonjour! Comment puis
Bonjour! Comment puis-
Bonjour! Comment puis-je
Bonjour! Comment puis-je vous
Bonjour! Comment puis-je vous aider
Bonjour! Comment puis-je vous aider aujourd
Bonjour! Comment puis-je vous aider aujourd'hui
Bonjour! Comment puis-je vous aider aujourd'hui en
Bonjour! Comment puis-je vous aider aujourd'hui en mat
Bonjour! Comment puis-je vous aider aujourd'hui en matière
Bonjour! Comment puis-je vous aider aujourd'hui en matière de
Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise
Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en
Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en base
Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en base de
Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en base de TV
Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en base de TVA
Bonjour! Comment puis

KeyboardInterrupt: ignored

In [18]:
print(history)

[{'role': 'user', 'content': 'salut'}, {'role': 'assistant', 'content': "Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en base de TVA et de formalités liées à la création d'entreprise ?"}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': ''}, {'role': 'user', 'content': 'hello'}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': "Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en base de TVA et de formalités liées à la création d'entreprise ?"}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': ''}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': "Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en base de TVA et de formalités liées à la création d'entreprise ?"}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': "Bonjour! Comment puis-je vous aider aujourd'hui en matière de franchise en base de TVA et de fo